<a href="https://colab.research.google.com/github/Alperenlcr/Python_and_Data_Science_Bootcamp/blob/main/Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Veri Kümesi Hakkında:


Bu veri seti, 51 odada kullanılan 255 sensör zaman serisinden toplanmıştır. Bir binadaki bir odanın fiziksel özelliklerindeki kalıpları araştırmak için kullanılabilir. Ayrıca, Nesnelerin İnterneti (IoT), sensör füzyon ağı veya zaman serisi görevleriyle ilgili deneyler için de kullanılabilir. Bu veri seti, hem denetimli (sınıflandırma ve regresyon) hem de denetimsiz öğrenme (kümeleme) görevleri için uygundur.


Her odada 5 tür ölçüm bulunur:
- CO2 konsantrasyonu,
- nem: oda havası nemi,
- sıcaklık: oda sıcaklığı,
- ışık: parlaklık,
- PIR: PIR hareket sensörü verileri


Veriler, 23 Ağustos 2013 Cuma gününden 31 Ağustos 2013 Cumartesi gününe kadar bir haftalık bir süre boyunca toplandı. PIR hareket sensörü her 10 saniyede bir, geri kalan sensörler ise her 5 saniyede bir örneklendi. Her dosya, zaman damgalarını ve sensörden gelen gerçek okumaları içerir.


Pasif kızılötesi sensör (PIR sensörü), görüş alanındaki nesnelerden yayılan kızılötesi (IR) ışığı ölçen ve bir odadaki doluluğu ölçen elektronik bir sensördür. PIR verilerinin yaklaşık %6'sı sıfır değildir ve odanın dolu olduğunu gösterir. PIR verilerinin kalan %94'ü sıfırdır ve boş bir oda olduğunu gösterir.


Yapılacak İşlemler


Klasörler biçiminde ayrılmış olan odaların verileri oda numarası bazlı olacak şekilde csv olarak klasör içinde verilmiştir.
Bu csv dosyaları ortak bir zaman diliminde (veri setinde unix time olarak verilmiştir) olacak şekilde birleştirerek tek bir tablo haline getirmeniz bekleniyor.
Boş gelen verileri silerek işlemlere devam edebilirsiniz.
Veri setini train ve test veri setleri şeklinde ayırıp modelleri o şekilde kurgulamasınız. Ayırma oranını train %70 ve test %30 olacak şekilde yapabilirsiniz.
Odada aktivite olup olmadığını tespit etmek için Sınıflandırma yöntemlerinin kullanılması beklenmektedir.
3 tane Sınıflandırma (KNN, Logistik Regresyon, Karar Ağaçları) modeli oluşturularak sonuçları karşılaştırmanız beklenmektedir.
Modeller oluştururken sınayacağınız veri setinden en doğru sonucu elde etmek için k-Fold Cross Validation yöntemini kullanın.


In [ ]:
from google.colab import drive
import pandas as pd
from glob import glob
import numpy as np

drive.mount('/content/drive/')
rooms = glob("/content/drive/MyDrive/sensors_dataset/*/", recursive = True)
room_names = [x.split('/')[-2] for x in rooms]
na_vals = ['nan', 'NaN', ' ', ' missing', 'Missing', 'naN', None]     # a list containing all the values to be deemed as null values

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# room separated
data_room_separated = {}
for i in range(len(room_names)):
  data_room_separated[room_names[i]]=pd.merge(
      pd.merge(pd.read_csv('{}co2.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'co2']),
               pd.read_csv('{}humidity.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'humidity']),
               on='time'),
      pd.merge(
          pd.merge(pd.read_csv('{}light.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'light']),
                   pd.read_csv('{}temperature.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'temperature']),
                   on='time'),
          pd.read_csv('{}pir.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'pir']),
          on='time'),
      on='time'
  )
data_room_separated

{'734':             time    co2  humidity  light  temperature   pir
 0     1377299126  384.0     47.47  311.0        24.80  17.0
 1     1377415711  314.0     54.04    4.0        23.56   0.0
 2     1377421672  313.0     56.51    4.0        23.22   0.0
 3     1377438542  310.0     59.43    6.0        22.76   0.0
 4     1377438761  304.0     59.46    6.0        22.77   0.0
 ...          ...    ...       ...    ...          ...   ...
 2415  1377966427  300.0     56.79   14.0        22.64   0.0
 2416  1377967463  313.0     56.63   13.0        22.68   0.0
 2417  1377968087  313.0     56.63   13.0        22.70   0.0
 2418  1377971180  320.0     56.70   15.0        22.85   0.0
 2419  1377972927  305.0     56.51   13.0        22.96   0.0
 
 [2420 rows x 6 columns],
 '776':             time    co2  humidity  light  temperature  pir
 0     1377299125  655.0     48.90  105.0        24.60  0.0
 1     1377299130  663.0     48.94  105.0        24.61  0.0
 2     1377300174  671.0     48.90  103.0     

In [ ]:
liste = []
for i in range(len(room_names)):
  co2=pd.read_csv('{}co2.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'co2'])
  co2['time'] = pd.to_datetime(co2['time'],unit='s')
  co2 = co2.set_index('time').resample('T').mean().reset_index()

  humidity=pd.read_csv('{}humidity.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'humidity'])
  humidity['time'] = pd.to_datetime(humidity['time'],unit='s')
  humidity = humidity.set_index('time').resample('T').mean().reset_index()

  light=pd.read_csv('{}light.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'light'])
  light['time'] = pd.to_datetime(light['time'],unit='s')
  light = light.set_index('time').resample('T').mean().reset_index()

  temperature=pd.read_csv('{}temperature.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'temperature'])
  temperature['time'] = pd.to_datetime(temperature['time'],unit='s')
  temperature = temperature.set_index('time').resample('T').mean().reset_index()

  pir=pd.read_csv('{}pir.csv'.format(rooms[i]), na_values=na_vals, usecols=[0, 1], names=['time', 'pir'])
  pir['time'] = pd.to_datetime(pir['time'],unit='s')
  pir = pir.set_index('time').resample('T').mean().reset_index()
  pir['pir'] = np.where(pir['pir'] > 0, 1, pir['pir'])
  data = pd.merge(
      pd.merge(co2, humidity, on='time'),
      pd.merge(
          pd.merge(light, temperature, on='time'), pir, on='time'),
      on='time'
  )
  data.insert(0, "Room_name", room_names[i], True)
  data.dropna()
  liste.append(data)
df = pd.concat(liste)
df

,Room_name,time,co2,humidity,light,temperature,pir
0,734,2013-08-23 23:05:00,383.000000,47.470000,312.250000,24.797500,1.0
1,734,2013-08-23 23:06:00,389.333333,47.473333,314.583333,24.800000,1.0
2,734,2013-08-23 23:07:00,388.250000,47.470000,317.750000,24.814167,1.0
3,734,2013-08-23 23:08:00,388.500000,47.470000,319.500000,24.816667,1.0
4,734,2013-08-23 23:09:00,387.416667,47.476667,321.333333,24.824167,0.0
...,...,...,...,...,...,...,...
11234,424,2013-08-31 18:19:00,386.666667,56.955000,3.500000,23.041667,0.0
11235,424,2013-08-31 18:20:00,387.166667,56.952500,4.250000,23.042500,0.0
11236,424,2013-08-31 18:21:00,395.416667,56.955000,3.583333,23.043333,0.0
11237,424,2013-08-31 18:22:00,388.666667,56.960000,3.500000,23.045833,0.0


In [ ]:
# Split the dataframe into two parts: one with the 'pir' column and another without the 'pir' column.
df2=df.dropna()
data = df2[['co2', 'humidity', 'light', 'temperature']]
label = df2['pir']

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

data_train, data_test, label_train, label_test = train_test_split(data, label, test_size=0.3, random_state=42)

In [ ]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_train_scaled = scaler.fit_transform(data_train)
data_test_scaled = scaler.transform(data_test)

In [ ]:
# RUN BELOW INSTEAD OF THIS BLOCK

# Train a KNN classifier on the training data
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

knn = HistGradientBoostingClassifier()
knn.fit(data_train_scaled, label_train)

accuracy = knn.score(data_test_scaled, label_test)
print("Accuracy:", accuracy)

HistGradientBoostingClassifier()

In [ ]:
# Training the K-NN model on the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(data_train_scaled, label_train)

KNeighborsClassifier()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# Fit KNN model
classifier.fit(data_train_scaled, label_train)

# Predict on the test data
label_pred = classifier.predict(data_test_scaled)

# Compute cross-validation scores
cv_scores = cross_val_score(classifier, data_train_scaled, label_train, cv=5)

In [ ]:
# Print mean and standard deviation of cross-validation scores
print("Cross-validation scores: {}".format(cv_scores))
print("Mean cross-validation score: {:.2f}".format(cv_scores.mean()))
print("Standard deviation of cross-validation scores: {:.2f}".format(cv_scores.std()))

# Print classification report
print("Classification Report:\n", classification_report(label_test, label_pred))

Cross-validation scores: [0.97443447 0.97435074 0.97368091 0.97440656 0.9749229 ]
Mean cross-validation score: 0.97
Standard deviation of cross-validation scores: 0.00
Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.99    144828
         1.0       0.83      0.72      0.77      8727

    accuracy                           0.98    153555
   macro avg       0.90      0.86      0.88    153555
weighted avg       0.97      0.98      0.97    153555



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression()
lr.fit(data_train_scaled, label_train)

# Compute cross-validation scores
cv_scores = cross_val_score(lr, data_train_scaled, label_train, cv=5)

prediction = lr.predict(data_test_scaled)

In [ ]:
# Print mean and standard deviation of cross-validation scores
print("Cross-validation scores: {}".format(cv_scores))
print("Mean cross-validation score: {:.2f}".format(cv_scores.mean()))
print("Standard deviation of cross-validation scores: {:.2f}".format(cv_scores.std()))

# Print classification report
print(classification_report(label_test, prediction))

Cross-validation scores: [0.95145062 0.95224605 0.95186927 0.95129712 0.95209255]
Mean cross-validation score: 0.95
Standard deviation of cross-validation scores: 0.00
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98    144828
         1.0       0.75      0.23      0.35      8727

    accuracy                           0.95    153555
   macro avg       0.85      0.61      0.66    153555
weighted avg       0.94      0.95      0.94    153555



In [ ]:
###### Single Decision Tree CLASSIFICATION in Python #######
from sklearn import tree
from sklearn.metrics import classification_report

#choose from different tunable hyper parameters
clf = tree.DecisionTreeClassifier(max_depth=4,criterion='entropy')
 
# Compute cross-validation scores
cv_scores = cross_val_score(clf, data_train_scaled, label_train, cv=5)
 
#Creating the model on Training Data
DTree=clf.fit(data_train,label_train)
prediction=DTree.predict(data_test)

In [ ]:
# Print mean and standard deviation of cross-validation scores
print("Cross-validation scores: {}".format(cv_scores))
print("Mean cross-validation score: {:.2f}".format(cv_scores.mean()))
print("Standard deviation of cross-validation scores: {:.2f}".format(cv_scores.std()))

# Print classification report
print(classification_report(label_test, prediction))

Cross-validation scores: [0.95220419 0.95143667 0.95174367 0.95071101 0.95399043]
Mean cross-validation score: 0.95
Standard deviation of cross-validation scores: 0.00
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.97    144828
         1.0       0.60      0.44      0.51      8727

    accuracy                           0.95    153555
   macro avg       0.79      0.71      0.74    153555
weighted avg       0.95      0.95      0.95    153555

